### Importing necessary libraries

In [1]:
!pip install opendatasets

In [2]:
!pip install --upgrade google-api-python-client

In [3]:
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [4]:
!pip install isodate

In [5]:
import opendatasets as od
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
import numpy as np
import statistics as st
import plotly.express as px
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from IPython.display import JSON
import isodate


warnings.filterwarnings('ignore')

### Loading the dataset from the source

In [6]:
dataset = 'https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset?select=US_youtube_trending_data.csv'

final_dir = '\\'.join(os.getcwd().split('\\')[:-1])

In [7]:
od.download(dataset, data_dir=final_dir)

final_dir += '\\youtube-trending-video-dataset\\US_youtube_trending_data.csv'

Skipping, found downloaded files in "C:\Users\timbe\Final Project\youtube-trending-video-dataset" (use force=True to force download)


In [8]:
data = pd.read_csv(final_dir)

data.head(3)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...


### Understanding the data

In [9]:
# Checking null values

def null_values(df):
    temp = df.isna().sum()
    temp_1 = round(temp * 100 / df.shape[0], 2)
    
    return pd.DataFrame((temp, temp_1), index = ['Count', 'Percentage']).T.sort_values('Count', ascending = False)


null_df = null_values(data).reset_index().rename({'index':'Column_name'}, axis =1)
null_df

,Column_name,Count,Percentage
0,description,4048.0,2.18
1,video_id,0.0,0.00
2,title,0.0,0.00
3,publishedAt,0.0,0.00
4,channelId,0.0,0.00
5,channelTitle,0.0,0.00
6,categoryId,0.0,0.00
7,trending_date,0.0,0.00
8,tags,0.0,0.00
9,view_count,0.0,0.00


We can see that there are around 10% missing values in `description` column
- We are keeping the `description` column as it is while analysis and perform the null value treatment during model building.

In [10]:
# Checking the shape of the DataFrame
data.shape

(185990, 16)

In [11]:
# Let's check the unique video_ids in the `video_id` column
data['video_id'].nunique()

34066

- There are only 34066 unique videos out of 185990 video ids.
- That means there might be some duplicate data. Let's check if there is any.

In [12]:
# This code shows an example of how a randomly chosen video_id has continuous trending_dates

example_id = data['video_id'].value_counts().sort_values(ascending = False).index[np.random.randint(10,20)]

data[data['video_id'] == example_id]

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
136823,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-23T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,4485049,121233,0,6910,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
137043,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-24T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,5976378,154258,0,8603,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
137257,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-25T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,6935854,177291,0,9578,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
137482,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-26T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,7874969,200501,0,10767,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
137669,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-27T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,8642711,221186,0,11722,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
137879,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-28T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,9287850,237943,0,12292,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
138077,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-29T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,9991118,255320,0,12863,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
138265,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-06-30T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,10445646,268538,0,13228,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
138481,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-07-01T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,10828819,280729,0,13603,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...
138688,ZD0doCSkBY8,Golden Buzzer: Mayyas' Breathtaking Audition C...,2022-06-22T02:00:04Z,UCT2X19JJaJGUN7mrYuImANQ,America's Got Talent,24,2022-07-02T00:00:00Z,Simon Cowell|Terry Crews|America's Got Talent|...,11248262,294031,0,14005,https://i.ytimg.com/vi/ZD0doCSkBY8/default.jpg,False,False,This audition from Mayyas was so breathtaking ...


- POOJA(observations)

- We also observed that the data has certain important columns missing in it. Such as `VideoDuration` and `Comments` of video which are very useful for further analysis.
- We used `video_id` to extract the data from `YOUTUBE API`.
- The link for the Youtube API is: https://developers.google.com/youtube/v3/quickstart/python

### Extracting `Duration` and `Comments` from Youtube API

In [13]:
# First let's get unique video ids from the video_id column in dataframe

videos_list = data['video_id'].unique()

In [14]:
# The API_KEY is not displayed for security purposes

api_key = "*********************************"

In [15]:
# This fuction is used to extract 'Duration' and 'Comments' of a provided video_id

def get_video_info(vid_id, api_key):
    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=api_key)

    request = youtube.videos().list(
        part="snippet,contentDetails,statistics,status",
        id=vid_id
    )
    
    response = request.execute()
    
    info_temp = []
    info_temp.append(vid_id)
    info_temp.append(response['items'][0]['contentDetails']['duration'])
    info_temp.append(response['items'][0]['status']['madeForKids'])
    
    youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = api_key)

    request = youtube.commentThreads().list(
        part="snippet,replies",
        videoId=vid_id,
        textFormat = "plainText",
        order = "relevance"
    )
    response_comments = request.execute()
    
    info_temp.append([response_comments['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal'] for i in range(20)])
    
    
    return info_temp



# This function is used to create a dataframe from the list of video_ids in video_list and save it in the repository.
# Start and end number is the range of video_ids you want the data for.

def data_to_csv(videos_list, start_number, end_number):
    print("Code Running")
    video_info = []
    deleted_videos = []
    
    for vid_id in videos_list[start_number : end_number]:
        try:
            video_info.append(get_video_info(vid_id, api_key))
        except:
            deleted_videos.append(vid_id)
    
    print("Number of videos extracted =", len(video_info))
    print("Number of video deleted =", len(deleted_videos))
    
    df = pd.DataFrame(video_info, columns = ['video_id', 'Duration', 'madeForKids', 'Comments'])
    df.to_csv('./YoutubeDataFiles/Data_' + str(start_number) + '_' + str(end_number) + '.csv')
    
    
# Saving the data to a .csv file by calling the funciton

#data_to_csv(videos_list, 0, 5000)

- Although it was possible to extract all the video_ids at once. It would cost money if you were to extract more than 5000.
- Hence, we used different API_KEYS and various start & end_numbers to save the files in the repository.
- The files are stored in `./YoutubeDataFiles` directory.
- The .csv files follow the convention `Data_startnumber_endnumber.csv`.

In [16]:
# Let's read the 'extracted files' in and merge them to merge with original Dataframe

extracted_df = pd.concat([pd.read_csv('./YoutubeDataFiles/' + file_csv, index_col = 0) for file_csv in os.listdir('./YoutubeDataFiles')],
                          ignore_index = True)

extracted_df.columns = ['video_id', 'videoDuration', 'madeForKids', 'Comments']

extracted_df.head()

,video_id,videoDuration,madeForKids,Comments
0,3C66w5Z0ixs,PT18M38S,False,"['*Whether she moved on, whether she’s been se..."
1,M9Pmf9AB4Mo,PT2M18S,False,"['She was fighting two squads and got flanked,..."
2,J78aPJ3VyNs,PT22M10S,False,"[""Let's see what memes you cooked up while I w..."
3,kXLn3HkpjaA,PT1M1S,False,"['Which 2020 XXL Freshman is your favorite?', ..."
4,VIUo6yapDbc,PT37M20S,False,['WE’RE SO OBSESSED!!!! Thank you guys times a...


In [17]:
# Checking the shape of the extracted dataset

extracted_df.shape

(31310, 4)

### Modifying the Original Dataframe

In [18]:
# Checking column name and values in the dataframe

data.head(1)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...


In [19]:
# Checking the datatypes of each column

data.dtypes

video_id             object
title                object
publishedAt          object
channelId            object
channelTitle         object
categoryId            int64
trending_date        object
tags                 object
view_count            int64
likes                 int64
dislikes              int64
comment_count         int64
thumbnail_link       object
comments_disabled      bool
ratings_disabled       bool
description          object
dtype: object

Let's make the following changes to the Original Dataframe in 4 major steps:

1. Drop `channelId` column as it is not necessary for the analysis and prediction.
<br>

1. Convert `publishedAt` and `trending_date` to Pandas_Datetime object. Map each category_id to respective category in `categroty_id` column. This is done using `US_category_id.json` file from the source file.
<br>

2. Groupby `video_id` column and apply the following functions to the rest of the columns:
- `title` - <b>Function : MODE</b> - because title is same for each video_id.
- `publishedAT` - <b>Function : MODE</b> - although published date is same for each video_id. Let's consider minimum value.
- `channelTitle` - <b>Function : MODE</b> - because channel title is same for each video_id.
- `categroryId` - <b>Function : MODE</b> - because categoryId is same for each video_id.
- `trending_date` - <b>Function : [min(trending_date), max(trending_date)]</b> - this is because we can extract features like `How many days video took to trend` and `How many days the video have been trending`.
- `tags` - <b>Function : MODE</b> - because tags are same for each video_id.
- `likes` - <b>Function : [min(likes), max(likes)]</b> - this is because we can extract features like `How many likes have be increased during the period`.
- `comments_disabled` - <b>Function : MODE</b> - because comments_disabled_flag is same for each video_id.
- `ratings_disabled` - <b>Function : MODE</b> - because ratings_disabled_flag is same for each video_id.
<br>
<br>
       
4. Drop `dislikes` column as the dislikes has been discontinued from November 2021.
Reference: https://www.google.com/search?q=when+did+youtube+remove+dislikes&rlz=1C1UEAD_enUS1037US1037&oq=when+did+youtube+remove+&aqs=chrome.0.0i512j69i57j0i512l6j0i22i30l2.6954j1j4&sourceid=chrome&ie=UTF-8



#### Step - 1

In [20]:
# Dropping channelId

data.drop(['channelId'], axis = 1, inplace = True)

# Validating the above code

data.head(3)

,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...


#### Step - 2

- `publishedAt`, `trending_date` are object type. Let's convert it to Datetime format.

In [21]:
# Converting 'publishedAt' and 'trending_date' to datetime objects

data[['publishedAt', 'trending_date']] = data[['publishedAt', 'trending_date']].apply(lambda x: pd.to_datetime(x))

# Validating the above changes

data.dtypes[['publishedAt', 'trending_date']]

publishedAt      datetime64[ns, UTC]
trending_date    datetime64[ns, UTC]
dtype: object

We can see that `categoryId` column has id's of repective categories. We can access the description of categoryId from the `US_category_id.json` file.

In [22]:
# Let's import US_category_id.json file and map the category id's respectively

category_path = '\\'.join(final_dir.split('\\')[:-1]) + '\\US_category_id.json'


# Creating a dictionary object which stores the category id and its respective category
category_dict = {}

with open(category_path, 'r') as file:
    json_data = json.load(file)
    for item in json_data['items']:
        category_dict[int(item['id'])] = item['snippet']['title']
    
data['categoryId'] = data['categoryId'].apply(lambda x: category_dict[x])

# Validating the above code
data['categoryId'].head()

0    People & Blogs
1            Gaming
2     Entertainment
3             Music
4     Howto & Style
Name: categoryId, dtype: object

#### Step - 3 and 4

In [23]:
# This function converts list to min_max_list as per conditions specified in the above cell

def column_start_end(x: list) -> list:
    return([min(x), max(x)])


# Grouping by 'video_id' and aggregating using functions as specified in the above cell

modified_df = data.groupby('video_id').agg({'title':st.mode, 'publishedAt':np.min, 'channelTitle':st.mode, 'categoryId':st.mode,
              'trending_date': column_start_end, 'tags': st.mode, 'likes': column_start_end, 'view_count': column_start_end,
                'comments_disabled': st.mode, 'ratings_disabled': st.mode}).reset_index()

modified_df.head(2)

,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,likes,view_count,comments_disabled,ratings_disabled
0,--14w5SOEUs,Migos - Avalanche (Official Video),2021-06-10 16:00:00+00:00,MigosVEVO,Music,"[2021-06-11 00:00:00+00:00, 2021-06-15 00:00:0...",Migos|Avalanche|Quality|Control|Music/Motown|R...,"[122830, 262692]","[1076217, 6823249]",False,False
1,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,2022-03-09 23:19:08+00:00,jf.okay,Entertainment,"[2022-03-11 00:00:00+00:00, 2022-03-15 00:00:0...",[None],"[16481, 17290]","[466980, 538485]",False,False


In [24]:
modified_df.shape

(34066, 11)

The resulting dataframe has 34066 rows and 10 columns.

Let's make the following changes to the resulting dataframe in 8 steps:
1. Create new columns `trending_date_start` & `trending_date_end` from `trending_date` column.
2. Create new columns `likes_start` & `likes_end` from `likes` column.
3. Create new columns `view_count_start` & `view_count_end` from `view_count` column.
3. `tags` have `[None]` values in it. Converting them to Null values.
4. Create new column `tagsCount` from `tags` column. Which indicates the number of tags used in particular video.
5. Create new column `hoursTakenToTrend` from `trending_date_start` & `publishedAt` columns.
6. Create new column `trendingDaysDuration` from `trending_date_end` & `trending_date_start` columns.
7. Drop columns `trending_date`, `likes`, `publishedAt`, `view_count` as they are no longer required.

#### Step - 1

In [25]:
# Create new columns trending_date_start & trending_date_end from trending_date column.

modified_df['trending_date_start'] = modified_df['trending_date'].apply(lambda x: min(x))
modified_df['trending_date_end'] = modified_df['trending_date'].apply(lambda x: max(x))

#### Step - 2

In [26]:
# Create new columns likes_start & likes_end from likes column.

modified_df['likes_start'] = modified_df['likes'].apply(lambda x: min(x))
modified_df['likes_end'] = modified_df['likes'].apply(lambda x: max(x))

#### Step - 3

In [27]:
# Create new columns `view_count_start` & `view_count_end` from `view_count` column.

modified_df['view_count_start'] = modified_df['view_count'].apply(lambda x: min(x))
modified_df['view_count_end'] = modified_df['view_count'].apply(lambda x: max(x))

#### Step - 4

In [28]:
# tags have [None] values in it. Converting them to Null values.

modified_df['tags'] = modified_df['tags'].apply(lambda x: x if x!= '[None]' else np.nan)

#### Step - 5

In [29]:
# Create new column tagsCount from tags column. Which indicates the number of tags used in particular video.

modified_df['tagCount'] = modified_df['tags'].apply(lambda x: 0 if type(x) == float else len(list(x.split('|'))))

#### Step - 6

In [30]:
# Create new column hoursTakenToTrend from trending_date_start & publishedAt columns.

modified_df['hoursTakenToTrend'] = round((modified_df['trending_date_start'] - modified_df['publishedAt']).dt.seconds/(60*60), 1)

#### Step - 7

In [31]:
# Create new column trendingDaysDuration from trending_date_end & trending_date_start columns.

modified_df['trendingDaysDuration'] = (modified_df['trending_date_end'] - modified_df['trending_date_start']).dt.days

#### Step - 8

In [32]:
# Drop columns trending_date, likes, publishedAt, view_count as they are no longer required.

modified_df.drop(['trending_date', 'likes', 'publishedAt', 'view_count'], axis = 1, inplace = True)

In [33]:
# Displaying the dataframe after all the changes

modified_df.head(3)

,video_id,title,channelTitle,categoryId,tags,comments_disabled,ratings_disabled,trending_date_start,trending_date_end,likes_start,likes_end,view_count_start,view_count_end,tagCount,hoursTakenToTrend,trendingDaysDuration
0,--14w5SOEUs,Migos - Avalanche (Official Video),MigosVEVO,Music,Migos|Avalanche|Quality|Control|Music/Motown|R...,False,False,2021-06-11 00:00:00+00:00,2021-06-15 00:00:00+00:00,122830,262692,1076217,6823249,8,8.0,4
1,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,jf.okay,Entertainment,NaN,False,False,2022-03-11 00:00:00+00:00,2022-03-15 00:00:00+00:00,16481,17290,466980,538485,0,0.7,4
2,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,Television Academy,Entertainment,NaN,False,False,2021-09-21 00:00:00+00:00,2021-09-25 00:00:00+00:00,6299,8029,527742,682609,0,22.9,4


In [34]:
# Displaying shape of Dataframe after all the changes

modified_df.shape

(34066, 16)

In [35]:
# Displaying the datatypes of the resultant Dataframe

modified_df.dtypes

video_id                             object
title                                object
channelTitle                         object
categoryId                           object
tags                                 object
comments_disabled                      bool
ratings_disabled                       bool
trending_date_start     datetime64[ns, UTC]
trending_date_end       datetime64[ns, UTC]
likes_start                           int64
likes_end                             int64
view_count_start                      int64
view_count_end                        int64
tagCount                              int64
hoursTakenToTrend                   float64
trendingDaysDuration                  int64
dtype: object

### Let's merge `modified_df` and `extracted_df`

In [36]:
# Merging 'modified_df' & 'extracted_df'

final_df = modified_df.merge(extracted_df, on = 'video_id', how = 'inner')

# Validating the above

final_df.head(2)

,video_id,title,channelTitle,categoryId,tags,comments_disabled,ratings_disabled,trending_date_start,trending_date_end,likes_start,likes_end,view_count_start,view_count_end,tagCount,hoursTakenToTrend,trendingDaysDuration,videoDuration,madeForKids,Comments
0,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,jf.okay,Entertainment,NaN,False,False,2022-03-11 00:00:00+00:00,2022-03-15 00:00:00+00:00,16481,17290,466980,538485,0,0.7,4,PT17S,False,['Reminds me of the “ how do you know if someo...
1,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,Television Academy,Entertainment,NaN,False,False,2021-09-21 00:00:00+00:00,2021-09-25 00:00:00+00:00,6299,8029,527742,682609,0,22.9,4,PT5M9S,False,"[""I get that she isn't a nobody, but to see so..."


In [37]:
# Checking the shape of the resulting dataset

final_df.shape

(31310, 19)

`videoDuration` column is in ISO Date format. Let's convert it into seconds

In [38]:
# Converting time in 'videoDuration' column to Seconds

final_df['videoDuration'] = final_df['videoDuration'].apply(lambda x: int(isodate.parse_duration(x).total_seconds()) )

# Validating the above changes

final_df.dtypes['videoDuration']

dtype('int64')

In [39]:
# Checking the Dataframe

final_df.head(2)

,video_id,title,channelTitle,categoryId,tags,comments_disabled,ratings_disabled,trending_date_start,trending_date_end,likes_start,likes_end,view_count_start,view_count_end,tagCount,hoursTakenToTrend,trendingDaysDuration,videoDuration,madeForKids,Comments
0,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,jf.okay,Entertainment,NaN,False,False,2022-03-11 00:00:00+00:00,2022-03-15 00:00:00+00:00,16481,17290,466980,538485,0,0.7,4,17,False,['Reminds me of the “ how do you know if someo...
1,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,Television Academy,Entertainment,NaN,False,False,2021-09-21 00:00:00+00:00,2021-09-25 00:00:00+00:00,6299,8029,527742,682609,0,22.9,4,309,False,"[""I get that she isn't a nobody, but to see so..."


## Exploratory Data Analysis

In [40]:
modified_df.columns

Index(['video_id', 'title', 'channelTitle', 'categoryId', 'tags',
       'comments_disabled', 'ratings_disabled', 'trending_date_start',
       'trending_date_end', 'likes_start', 'likes_end', 'view_count_start',
       'view_count_end', 'tagCount', 'hoursTakenToTrend',
       'trendingDaysDuration'],
      dtype='object')

In [41]:
# Let's check the correlation between the numerical columns

plt.figure(figsize = (10, 8))
sns.heatmap(data[['view_count', 'likes', 'dislikes', 'comment_count', 'daysTakenToTrend', 'tagCount']].corr(), linewidths=.5, annot=True, cmap='coolwarm')
plt.show()

KeyError: "['daysTakenToTrend', 'tagCount'] not in index"

<Figure size 1000x800 with 0 Axes>

- `view_count` and `likes` are highly correlated. It is more likely that the video with more views has more likes.
- `comment_count` and `likes` are relatively highly correlated when compared to `comment_count` and `views`.
- `daysTakenToTrend` is not correlated to any feature. Which is interesting as it is impossible to correlate how many days the video will take to trend based on comment_count or dislikes or likes or view_count.

In [ ]:
######################### likes per view

In [ ]:
# Checking number of videos based on each Category

plt.figure(figsize = (10, 4))
sns.countplot(data['categoryId'], order = data['categoryId'].value_counts().sort_values(ascending = False).index)
plt.xticks(rotation = 90)
plt.show()

There are more `Entertainment` videos and least type is `Nonprofits & Activism`

In [ ]:
plt.figure(figsize = (10, 6))
sns.barplot(data = data, x = 'categoryId', y = 'likes',
            order = data.groupby('categoryId')['likes'].mean().sort_values(ascending = False).index, ci = 0)

plt.xticks(rotation = 90)
plt.show()

- `Pets & Animals` videos has most average likes and `New & Policts` videos has least average likes.

In [ ]:
plt.figure(figsize = (10, 6))
sns.barplot(data = data, x = 'categoryId', y = 'comment_count',
            order = data.groupby('categoryId')['comment_count'].mean().sort_values(ascending = False).index, ci = 0)

plt.xticks(rotation = 90)
plt.show()

`Music` videos has most average comment count and `Nonprofits & Activism` has least average comment count.

In [ ]:
plt.figure(figsize = (10, 6))
sns.barplot(data = data, x = 'categoryId', y = 'daysTakenToTrend',
            order = data.groupby('categoryId')['daysTakenToTrend'].mean().sort_values(ascending = False).index, ci = 0)

plt.xticks(rotation = 90)
plt.show()

It is interesting to note that `News & Politics` videos take less time to trend and `Music`, `Comedy` and `Pets & Animals` videos take more time to trend.

In [ ]:
sns.countplot(data['comments_disabled'])
plt.show()

In [ ]:
plt.figure(figsize = (10, 5))
sns.countplot(data[data['ratings_disabled'] == True]['categoryId'], 
              order = data[data['ratings_disabled'] == True].groupby('categoryId')['ratings_disabled'].count().sort_values(ascending = False).index)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.figure(figsize = (10, 5))
sns.countplot(data[data['tags'] == '[None]']['categoryId'], 
              order = data[data['tags'] == '[None]'].groupby('categoryId')['ratings_disabled'].count().sort_values(ascending = False).index)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# Let's extract weekday from the trendingdate
data['day'] = data['trending_date'].dt.day_name()

In [ ]:
# Let's plot number of trending videos for each day of the week
sns.countplot(data['day'])
plt.show()

In [ ]:
# Daily several videos trend but which video trends for the longest number of days will be the question.

In [ ]:
# Modify
plt.figure(figsize = (15,4))
sns.countplot(data = data, x = 'categoryId', hue = 'day')

In [ ]:
# categories that are taking minimum or moderate or maximum number of days to trend
(data.groupby('categoryId')['daysTakenToTrend'].var().sort_values()).plot.bar()

In [ ]:

data['like/dislike ratio'] = round(data['likes']/data['dislikes'], 2)

In [ ]:
# Modify
plt.figure(figsize = (8, 10))
sns.histplot(data = data, x = 'like/dislike ratio', y = 'categoryId')